In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Interim Data File Locations
interim_data = '../data/interim/'

In [3]:
df = pd.read_csv(interim_data + 'pos_neg_class_file.csv')
df.shape

(87550, 3)

In [4]:
df.sample(10)

,name_a,name_b,target
25461,Montserrat,Gerdale,0
46215,Natius,Joann,0
76133,Sid,Yul,0
51961,Christi,Joaninha,0
60609,Norman,Blagi,0
87209,Kentaro,Chloe,0
36050,Matteo,Agge,0
9106,Liz,Bet,1
59568,Xandra,Ovidiu,0
27328,Dolo,Cornelia,0


In [5]:
# Save output to interim data folder as csv file
df.to_csv(interim_data + 'eda_results.csv', index=False)